#Data Cleaning Pipeline for SmartMart

In [180]:
import pandas as pd
import numpy as np
import os

# Step 1: Load datasets

In [181]:
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
sales = pd.read_csv("sales_transactions.csv")
stock = pd.read_csv("stock_levels.csv")
stores = pd.read_csv("stores.csv")

# Step 2: Cleaning Functions

In [182]:
def clean_dataframe(df):
    """General cleaning: drop duplicates, strip spaces, standardize column names."""
    # Drop duplicates
    df = df.drop_duplicates()

    # Strip spaces from column names
    df.columns = df.columns.str.strip().str.lower()

    # Strip spaces from string columns
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].str.strip()

    return df

In [183]:
def fill_missing_values(df):
    """Handle missing values with simple strategies."""
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].fillna("Unknown")
        else:
            df[col] = df[col].fillna(df[col].median())  # median for numeric
    return df

In [184]:
def standardize_text(df, columns):
    """Standardize categorical text columns (capitalize first letter)."""
    for col in columns:
        if col in df.columns:
            df[col] = df[col].str.capitalize()
    return df

# Step 3: Apply Cleaning

In [185]:
# Customers
customers = clean_dataframe(customers)
customers = fill_missing_values(customers)
customers = standardize_text(customers, ["gender", "city", "loyalty_member"])

In [186]:
# Products
products = clean_dataframe(products)
products = fill_missing_values(products)
products = standardize_text(products, ["category", "subcategory"])

In [187]:
# Sales Transactions
sales = clean_dataframe(sales)
sales = fill_missing_values(sales)

In [188]:
# Convert date column to datetime (if exists)
if "transaction_date" in sales.columns:
    sales["transaction_date"] = pd.to_datetime(sales["transaction_date"])

/tmp/ipython-input-296878576.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sales["transaction_date"] = pd.to_datetime(sales["transaction_date"])


In [189]:
# Stock Levels
stock = clean_dataframe(stock)
stock = fill_missing_values(stock)

In [190]:
# Stores
stores = clean_dataframe(stores)
stores = fill_missing_values(stores)
stores = standardize_text(stores, ["city"])

# Step 4: Additional Preprocessing

In [191]:
if customers['customer_id'].dtype == 'object':
  customers['customer_id'] = customers['customer_id'].str.lstrip('CUST_').astype(int)
if customers["loyalty_member"].dtype != bool:
  customers['loyalty_member'] = customers['loyalty_member'].replace({'Yes': True, 'No': False})
  customers['loyalty_member'] = customers['loyalty_member'].astype(bool)
customers = pd.get_dummies(customers, columns=['gender', 'city'], dtype=bool)

/tmp/ipython-input-28362746.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  customers['loyalty_member'] = customers['loyalty_member'].replace({'Yes': True, 'No': False})


In [192]:
sales['customer_id'] = sales['customer_id'].replace('Unknown', np.nan)
if sales['customer_id'].dtype == 'object':
  sales['customer_id'] = sales['customer_id'].str.lstrip('CUST_').astype(float)
sales = pd.get_dummies(sales, columns=['store_id'], dtype=bool)

In [193]:
stock = pd.get_dummies(stock, columns=['store_id'], dtype=bool)

In [194]:
stores = pd.get_dummies(stores, columns=['store_id', 'store_format'], dtype=bool)

In [195]:
products = pd.get_dummies(products, columns=['category'], dtype=bool)
products = products.drop(columns=['product_name', 'unit_size'], axis=1)

# Step 5: Save processed datasets

In [196]:
output_dir = "/content/cleaned_datasets"
os.makedirs(output_dir, exist_ok=True)

In [197]:
customers.to_csv(f"{output_dir}/customers_clean.csv", index=False)
products.to_csv(f"{output_dir}/products_clean.csv", index=False)
sales.to_csv(f"{output_dir}/sales_transactions_clean.csv", index=False)
stock.to_csv(f"{output_dir}/stock_levels_clean.csv", index=False)
stores.to_csv(f"{output_dir}/stores_clean.csv", index=False)

In [198]:
print("All cleaned datasets saved in:", output_dir)

All cleaned datasets saved in: /content/cleaned_datasets
